## Codes for scrapping fbref

In [2]:
import requests 
import time
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
standings_url = "Https://fbref.com/en/comps/9/Premier-League-Stats"
data = requests.get(standings_url) 
soup = BeautifulSoup(data.text) 
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]
team_urls = [f"https://fbref.com{l}" for l in links]

In [9]:
all_matches = []

for team_url in team_urls:
    team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
    data = requests.get(team_url)
    matches = pd.read_html(data.text, match = "Scores & Fixtures")
    
    soup = BeautifulSoup(data.text)
    links = soup.find_all('a')
    links = [l.get("href") for l in links]
    links = [l for l in links if l and 'all_comps/passing/' in l]
    shooting_urls = [f"https://fbref.com{l}" for l in links]
    data = requests.get(shooting_urls[0])
    shooting = pd.read_html(data.text, match = "Passing")[0]
    shooting.columns = shooting.columns.droplevel()
    
    try:
        team_data = matches[0].merge(shooting[["Cmp", "Att", "TotDist", "PrgDist", "Ast", "xA", "KP", "1/3", "PPA", "CrsPA", "Prog"]], on="Date")
    except ValueError:
        continue
    team_data = team_data[team_data["Comp"] == "Premier League"]
    
    team_data["Season"] = "21/22"
    team_data["Team"] = team_name
    all_matches.append(team_data)
    
    time.sleep(1)

KeyError: 'Date'

In [5]:
match_df =pd.concat(all_matches)

In [6]:
match_df.columns = [c.lower() for c in match_df.columns]

In [7]:
match_df.to_csv("eplmatch_Shooting.csv")

In [8]:
match_df.to_excel(r"D:\OneDrive\Projects\EPL Data Exploration\Scrapping project\eplmatch_Shooting.xlsx")

## Exercise and details

In [1]:
import requests 

In [2]:
standings_url = "Https://fbref.com/en/comps/9/Premier-League-Stats"

In [3]:
data = requests.get(standings_url) 
# data consists of all of the html data from the web

In [4]:
from bs4 import BeautifulSoup

In [8]:
soup = BeautifulSoup(data.text) 
# Use beautifulsoup ability to turn all the data into text
soup

<!DOCTYPE html>
<html class="no-js" data-root="/home/fb/deploy/www/base" data-version="klecko-" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport"/>
<link href="https://d2p3bygnnzw9w3.cloudfront.net/req/202205101" rel="dns-prefetch"/>
<!-- Quantcast Choice. Consent Manager Tag v2.0 (for TCF 2.0) -->
<script async="true" type="text/javascript">
    (function() {
	var host = window.location.hostname;
	var element = document.createElement('script');
	var firstScript = document.getElementsByTagName('script')[0];
	var url = 'https://quantcast.mgr.consensu.org'
	    .concat('/choice/', 'XwNYEpNeFfhfr', '/', host, '/choice.js')
	var uspTries = 0;
	var uspTriesLimit = 3;
	element.async = true;
	element.type = 'text/javascript';
	element.src = url;
	
	firstScript.parentNode.insertBefore(element, firstScript);
	
	function makeStub() {
	    var TCF_LOCATOR_NAME = 

In [9]:
# We define the standing table by taking everything that falls 
#under the standing table.
standings_table = soup.select('table.stats_table')[0]


In [10]:
# use find_all to get all the 'a'tag from the HTML

links = standings_table.find_all('a')
links = [l.get("href") for l in links] 
links
# creating a list to store the information that has "href" in it.

['/en/squads/b8fd03ef/Manchester-City-Stats',
 '/en/players/e46012d4/Kevin-De-Bruyne',
 '/en/players/3bb7b8b4/Ederson',
 '/en/squads/822bd0ba/Liverpool-Stats',
 '/en/players/e342ad68/Mohamed-Salah',
 '/en/players/7a2e46a8/Alisson',
 '/en/squads/cff3d9bb/Chelsea-Stats',
 '/en/players/9674002f/Mason-Mount',
 '/en/players/33887998/Edouard-Mendy',
 '/en/squads/361ca564/Tottenham-Hotspur-Stats',
 '/en/players/92e7e919/Son-Heung-min',
 '/en/players/8f62b6ee/Hugo-Lloris',
 '/en/squads/18bb7c10/Arsenal-Stats',
 '/en/players/bc7dc64d/Bukayo-Saka',
 '/en/players/466fb2c5/Aaron-Ramsdale',
 '/en/squads/19538871/Manchester-United-Stats',
 '/en/players/dea698d9/Cristiano-Ronaldo',
 '/en/players/7ba6d84e/David-de-Gea',
 '/en/squads/7c21e445/West-Ham-United-Stats',
 '/en/players/79c84d1c/Jarrod-Bowen',
 '/en/players/9328b835/Lukasz-Fabianski',
 '/en/squads/a2d435b3/Leicester-City-Stats',
 '/en/players/45963054/Jamie-Vardy',
 '/en/players/53af52f3/Kasper-Schmeichel',
 '/en/squads/d07537b9/Brighton-and-

In [11]:
links = [l for l in links if '/squads/' in l]
# Only choosing those that are squad
links

['/en/squads/b8fd03ef/Manchester-City-Stats',
 '/en/squads/822bd0ba/Liverpool-Stats',
 '/en/squads/cff3d9bb/Chelsea-Stats',
 '/en/squads/361ca564/Tottenham-Hotspur-Stats',
 '/en/squads/18bb7c10/Arsenal-Stats',
 '/en/squads/19538871/Manchester-United-Stats',
 '/en/squads/7c21e445/West-Ham-United-Stats',
 '/en/squads/a2d435b3/Leicester-City-Stats',
 '/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats',
 '/en/squads/8cec06e1/Wolverhampton-Wanderers-Stats',
 '/en/squads/b2b47a98/Newcastle-United-Stats',
 '/en/squads/47c64c55/Crystal-Palace-Stats',
 '/en/squads/cd051869/Brentford-Stats',
 '/en/squads/8602292d/Aston-Villa-Stats',
 '/en/squads/33c895d4/Southampton-Stats',
 '/en/squads/d3fd31cc/Everton-Stats',
 '/en/squads/5bfb9659/Leeds-United-Stats',
 '/en/squads/943e8050/Burnley-Stats',
 '/en/squads/2abfe087/Watford-Stats',
 '/en/squads/1c781004/Norwich-City-Stats']

In [12]:
team_urls = [f"https://fbref.com{l}" for l in links]
team_urls

['https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats',
 'https://fbref.com/en/squads/822bd0ba/Liverpool-Stats',
 'https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats',
 'https://fbref.com/en/squads/361ca564/Tottenham-Hotspur-Stats',
 'https://fbref.com/en/squads/18bb7c10/Arsenal-Stats',
 'https://fbref.com/en/squads/19538871/Manchester-United-Stats',
 'https://fbref.com/en/squads/7c21e445/West-Ham-United-Stats',
 'https://fbref.com/en/squads/a2d435b3/Leicester-City-Stats',
 'https://fbref.com/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats',
 'https://fbref.com/en/squads/8cec06e1/Wolverhampton-Wanderers-Stats',
 'https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats',
 'https://fbref.com/en/squads/47c64c55/Crystal-Palace-Stats',
 'https://fbref.com/en/squads/cd051869/Brentford-Stats',
 'https://fbref.com/en/squads/8602292d/Aston-Villa-Stats',
 'https://fbref.com/en/squads/33c895d4/Southampton-Stats',
 'https://fbref.com/en/squads/d3fd31cc/Everton-Stats',
 'https://fbref.

In [10]:
team_url = team_urls[0]
# Choosing the first one of all the team urls (Man City)
team_url

'https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats'

In [11]:
data = requests.get(team_url)

In [12]:
import pandas as pd

matches = pd.read_html(data.text, match = "Scores & Fixtures")

In [13]:
matches[0]


,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes
0,2021-08-07,17:15,Community Shield,FA Community Shield,Sat,Neutral,L,0,1,Leicester City,NaN,NaN,57,NaN,Fernandinho,4-3-3,Paul Tierney,Match Report,NaN
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0,1,Tottenham,1.9,1.3,64,58262.0,Fernandinho,4-3-3,Anthony Taylor,Match Report,NaN
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5,0,Norwich City,2.7,0.1,67,51437.0,İlkay Gündoğan,4-3-3,Graham Scott,Match Report,NaN
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5,0,Arsenal,3.8,0.1,80,52276.0,İlkay Gündoğan,4-3-3,Martin Atkinson,Match Report,NaN
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1,0,Leicester City,2.9,0.8,61,32087.0,İlkay Gündoğan,4-3-3,Paul Tierney,Match Report,NaN
5,2021-09-15,20:00,Champions Lg,Group stage,Wed,Home,W,6,3,de RB Leipzig,2.1,0.6,51,38062.0,Rúben Dias,4-3-3,Serdar Gözübüyük,Match Report,NaN
6,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0,0,Southampton,1.1,0.4,63,52698.0,Fernandinho,4-3-3,Jonathan Moss,Match Report,NaN
7,2021-09-21,19:45,EFL Cup,Third round,Tue,Home,W,6,1,Wycombe,NaN,NaN,79,30959.0,Kevin De Bruyne,4-3-3,Robert Jones,Match Report,NaN
8,2021-09-25,12:30,Premier League,Matchweek 6,Sat,Away,W,1,0,Chelsea,1.7,0.3,60,40036.0,Rúben Dias,4-3-3,Michael Oliver,Match Report,NaN
9,2021-09-28,21:00,Champions Lg,Group stage,Tue,Away,L,0,2,fr Paris S-G,1.9,0.8,54,37350.0,Rúben Dias,4-3-3,Carlos del Cerro,Match Report,NaN


In [14]:
soup = BeautifulSoup(data.text)
soup

<!DOCTYPE html>
<html class="no-js" data-root="/home/fb/deploy/www/base" data-version="klecko-" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport"/>
<link href="https://d2p3bygnnzw9w3.cloudfront.net/req/202205232" rel="dns-prefetch"/>
<!-- Quantcast Choice. Consent Manager Tag v2.0 (for TCF 2.0) -->
<script async="true" type="text/javascript">
    (function() {
	var host = window.location.hostname;
	var element = document.createElement('script');
	var firstScript = document.getElementsByTagName('script')[0];
	var url = 'https://quantcast.mgr.consensu.org'
	    .concat('/choice/', 'XwNYEpNeFfhfr', '/', host, '/choice.js')
	var uspTries = 0;
	var uspTriesLimit = 3;
	element.async = true;
	element.type = 'text/javascript';
	element.src = url;
	
	firstScript.parentNode.insertBefore(element, firstScript);
	
	function makeStub() {
	    var TCF_LOCATOR_NAME = 

In [15]:
links = soup.find_all('a')

In [16]:
links = [l.get("href") for l in links]
links

['https://fbref.com/pt/squads/b8fd03ef/Manchester-City-Stats',
 'https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats',
 'https://fbref.com/fr/squads/b8fd03ef/Manchester-City-Stats',
 'https://fbref.com/it/squads/b8fd03ef/Manchester-City-Stats',
 'https://fbref.com/de/squads/b8fd03ef/Manchester-City-Stats',
 'https://fbref.com/es/squads/b8fd03ef/Manchester-City-Stats',
 'https://www.sports-reference.com/',
 'https://www.baseball-reference.com/',
 'https://www.pro-football-reference.com/',
 'https://www.sports-reference.com/cfb/',
 'https://www.basketball-reference.com/',
 'https://www.sports-reference.com/cbb/',
 'https://www.hockey-reference.com/',
 'https://fbref.com/',
 'https://www.sports-reference.com/blog/',
 'https://stathead.com/?utm_source=web&utm_medium=fb&utm_campaign=sr-nav-bar-top-link',
 'https://widgets.sports-reference.com/',
 'https://www.sports-reference.com/feedback/',
 'https://stathead.com/profile/?utm_source=web&utm_medium=fb&utm_campaign=sr-nav-bar-top-acco

In [17]:
links = [l for l in links if l and 'all_comps/passing/' in l]

In [18]:
links

['/en/squads/b8fd03ef/2021-2022/matchlogs/all_comps/passing/Manchester-City-Match-Logs-All-Competitions',
 '/en/squads/b8fd03ef/2021-2022/matchlogs/all_comps/passing/Manchester-City-Match-Logs-All-Competitions',
 '/en/squads/b8fd03ef/2021-2022/matchlogs/all_comps/passing/Manchester-City-Match-Logs-All-Competitions',
 '/en/squads/b8fd03ef/2021-2022/matchlogs/all_comps/passing/Manchester-City-Match-Logs-All-Competitions']

In [19]:
shooting_urls = [f"https://fbref.com{l}" for l in links]
shooting_urls

['https://fbref.com/en/squads/b8fd03ef/2021-2022/matchlogs/all_comps/passing/Manchester-City-Match-Logs-All-Competitions',
 'https://fbref.com/en/squads/b8fd03ef/2021-2022/matchlogs/all_comps/passing/Manchester-City-Match-Logs-All-Competitions',
 'https://fbref.com/en/squads/b8fd03ef/2021-2022/matchlogs/all_comps/passing/Manchester-City-Match-Logs-All-Competitions',
 'https://fbref.com/en/squads/b8fd03ef/2021-2022/matchlogs/all_comps/passing/Manchester-City-Match-Logs-All-Competitions']

In [20]:
data = requests.get(shooting_urls[0])
data

<Response [200]>

In [21]:
shooting = pd.read_html(data.text, match = "Passing")[0]

In [22]:
shooting

For Manchester City                                                      \
                 Date Time Comp Round Day Venue  Result   GF  GA Opponent   
0                 NaN  NaN  NaN   NaN NaN   NaN  41-9-8  150  50      NaN   

   ...  Long       Unnamed: 24_level_0 Unnamed: 25_level_0  \
   ...   Att  Cmp%                 Ast                  xA   
0  ...  5458  73.9                 102                77.3   

  Unnamed: 26_level_0 Unnamed: 27_level_0 Unnamed: 28_level_0  \
                   KP                 1/3                 PPA   
0                 676                2552                 726   

  Unnamed: 29_level_0 Unnamed: 30_level_0 Unnamed: 31_level_0  
                CrsPA                Prog        Match Report  
0                 100                2331                 NaN  

[1 rows x 32 columns]

In [32]:
shooting.columns = shooting.columns.droplevel()

In [33]:
shooting.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Dist,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG,Match Report
0,2021-08-07,17:15,Community Shield,FA Community Shield,Sat,Neutral,L,0,1,Leicester City,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,Match Report
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0,1,Tottenham,...,16.9,1.0,0,0,1.9,1.9,0.11,-1.9,-1.9,Match Report
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5,0,Norwich City,...,17.3,1.0,0,0,2.7,2.7,0.17,1.3,1.3,Match Report
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5,0,Arsenal,...,14.3,0.0,0,0,3.8,3.8,0.15,1.2,1.2,Match Report
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1,0,Leicester City,...,14.0,0.0,0,0,2.9,2.9,0.12,-1.9,-1.9,Match Report


In [34]:
shooting.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 26 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          58 non-null     object 
 1   Time          58 non-null     object 
 2   Comp          58 non-null     object 
 3   Round         58 non-null     object 
 4   Day           58 non-null     object 
 5   Venue         58 non-null     object 
 6   Result        59 non-null     object 
 7   GF            59 non-null     object 
 8   GA            59 non-null     object 
 9   Opponent      58 non-null     object 
 10  Gls           59 non-null     int64  
 11  Sh            59 non-null     int64  
 12  SoT           59 non-null     int64  
 13  SoT%          59 non-null     float64
 14  G/Sh          59 non-null     float64
 15  G/SoT         59 non-null     float64
 16  Dist          51 non-null     float64
 17  FK            51 non-null     float64
 18  PK            59 non-null     in

In [35]:
# using pandas merge function
# team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
team_data = matches[0].merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [36]:
team_data.shape

(58, 25)

In [37]:
team)

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2021-08-07,17:15,Community Shield,FA Community Shield,Sat,Neutral,L,0,1,Leicester City,...,4-3-3,Paul Tierney,Match Report,NaN,12,3,NaN,NaN,0,0
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0,1,Tottenham,...,4-3-3,Anthony Taylor,Match Report,NaN,18,4,16.9,1.0,0,0
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5,0,Norwich City,...,4-3-3,Graham Scott,Match Report,NaN,16,4,17.3,1.0,0,0
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5,0,Arsenal,...,4-3-3,Martin Atkinson,Match Report,NaN,25,10,14.3,0.0,0,0
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1,0,Leicester City,...,4-3-3,Paul Tierney,Match Report,NaN,25,8,14.0,0.0,0,0
5,2021-09-15,20:00,Champions Lg,Group stage,Wed,Home,W,6,3,de RB Leipzig,...,4-3-3,Serdar Gözübüyük,Match Report,NaN,16,8,15.7,0.0,1,1
6,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0,0,Southampton,...,4-3-3,Jonathan Moss,Match Report,NaN,16,1,15.7,1.0,0,0
7,2021-09-21,19:45,EFL Cup,Third round,Tue,Home,W,6,1,Wycombe,...,4-3-3,Robert Jones,Match Report,NaN,26,14,NaN,NaN,0,0
8,2021-09-25,12:30,Premier League,Matchweek 6,Sat,Away,W,1,0,Chelsea,...,4-3-3,Michael Oliver,Match Report,NaN,15,3,16.5,0.0,0,0
9,2021-09-28,21:00,Champions Lg,Group stage,Tue,Away,L,0,2,fr Paris S-G,...,4-3-3,Carlos del Cerro,Match Report,NaN,18,7,14.7,2.0,0,0


In [66]:
# years = list(range(2022,2020, -1))

In [67]:
# years

[2022]

In [52]:
all_matches = []

In [53]:
import requests 
import time
from bs4 import BeautifulSoup
all_matches = []
import pandas as pd

for team_url in team_urls:
    team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
    data = requests.get(team_url)
    matches = pd.read_html(data.text, match = "Scores & Fixtures")
    
    soup = BeautifulSoup(data.text)
    links = soup.find_all('a')
    links = [l.get("href") for l in links]
    links = [l for l in links if l and 'all_comps/shooting/' in l]
    shooting_urls = [f"https://fbref.com{l}" for l in links]
    data = requests.get(shooting_urls[0])
    shooting = pd.read_html(data.text, match = "Shooting")[0]
    shooting.columns = shooting.columns.droplevel()
    
    try:
        team_data = matches[0].merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
    except ValueError:
        continue
    team_data = team_data[team_data["Comp"] == "Premier League"]
    
    team_data["Season"] = "21/22"
    team_data["Team"] = team_name
    all_matches.append(team_data)
    
    time.sleep(1)

In [57]:
all_matches[1]

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt,Season,Team
0,2021-08-14,17:30,Premier League,Matchweek 1,Sat,Away,W,3,0,Norwich City,...,Match Report,NaN,19,6,16.4,1.0,0,0,21/22,Liverpool
1,2021-08-21,12:30,Premier League,Matchweek 2,Sat,Home,W,2,0,Burnley,...,Match Report,NaN,28,8,15.1,0.0,0,0,21/22,Liverpool
2,2021-08-28,17:30,Premier League,Matchweek 3,Sat,Home,D,1,1,Chelsea,...,Match Report,NaN,23,6,14.8,0.0,1,1,21/22,Liverpool
3,2021-09-12,16:30,Premier League,Matchweek 4,Sun,Away,W,3,0,Leeds United,...,Match Report,NaN,30,8,14.7,1.0,0,0,21/22,Liverpool
5,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,W,3,0,Crystal Palace,...,Match Report,NaN,25,10,15.2,0.0,0,0,21/22,Liverpool
7,2021-09-25,17:30,Premier League,Matchweek 6,Sat,Away,D,3,3,Brentford,...,Match Report,NaN,15,4,16.2,1.0,0,0,21/22,Liverpool
9,2021-10-03,16:30,Premier League,Matchweek 7,Sun,Home,D,2,2,Manchester City,...,Match Report,NaN,6,3,15.2,1.0,0,0,21/22,Liverpool
10,2021-10-16,12:30,Premier League,Matchweek 8,Sat,Away,W,5,0,Watford,...,Match Report,NaN,19,8,15.5,0.0,0,0,21/22,Liverpool
12,2021-10-24,16:30,Premier League,Matchweek 9,Sun,Away,W,5,0,Manchester Utd,...,Match Report,NaN,19,8,16.0,0.0,0,0,21/22,Liverpool
14,2021-10-30,15:00,Premier League,Matchweek 10,Sat,Home,D,2,2,Brighton,...,Match Report,NaN,14,3,19.7,0.0,0,0,21/22,Liverpool


In [58]:
len(all_matches)

20

In [59]:
match_df =pd.concat(all_matches)

In [60]:
match_df.columns = [c.lower() for c in match_df.columns]

In [61]:
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0,1,Tottenham,...,Match Report,NaN,18.0,4.0,16.9,1.0,0.0,0.0,21/22,Manchester City
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5,0,Norwich City,...,Match Report,NaN,16.0,4.0,17.3,1.0,0.0,0.0,21/22,Manchester City
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5,0,Arsenal,...,Match Report,NaN,25.0,10.0,14.3,0.0,0.0,0.0,21/22,Manchester City
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1,0,Leicester City,...,Match Report,NaN,25.0,8.0,14.0,0.0,0.0,0.0,21/22,Manchester City
6,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0,0,Southampton,...,Match Report,NaN,16.0,1.0,15.7,1.0,0.0,0.0,21/22,Manchester City
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,2022-04-30,15:00,Premier League,Matchweek 35,Sat,Away,L,0,2,Aston Villa,...,Match Report,NaN,10.0,3.0,20.7,0.0,0.0,0.0,21/22,Norwich City
39,2022-05-08,14:00,Premier League,Matchweek 36,Sun,Home,L,0,4,West Ham,...,Match Report,NaN,8.0,2.0,21.5,1.0,0.0,0.0,21/22,Norwich City
40,2022-05-11,19:45,Premier League,Matchweek 21,Wed,Away,L,0,3,Leicester City,...,Match Report,NaN,9.0,5.0,16.2,0.0,0.0,0.0,21/22,Norwich City
41,2022-05-15,14:00,Premier League,Matchweek 37,Sun,Away,D,1,1,Wolves,...,Match Report,NaN,11.0,2.0,13.4,0.0,0.0,0.0,21/22,Norwich City


In [62]:
match_df.to_csv("eplmatch.csv")

In [63]:
match_df.to_excel(r"D:\OneDrive\Projects\EPL Data Exploration\Scrapping project\eplmatch.xlsx")